In [1]:
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

In [4]:
@tf.function
def extract_matrix(f):
    lines = tf.strings.split(f, '\n')
    lines = tf.strings.split(lines, ' bags contain ')
    lines = lines.to_tensor()
    bagtypes = lines[:,0]
    n = tf.shape(bagtypes)[0]
    
    init = tf.lookup.KeyValueTensorInitializer(bagtypes, tf.range(n))
    index_map = tf.lookup.StaticHashTable(init, default_value=-1)
    sg_idx = index_map[tf.constant('shiny gold')]
    
    indices = tf.TensorArray(tf.int32, size=0, dynamic_size=True)
    updates = tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    
    ta_i = 0

    for i in tf.range(n):
        c = lines[i][1]
        c = tf.strings.split(c, ', ')
        for j in tf.range(tf.shape(c)[0]):
            e = c[j]
            e_split = tf.strings.split(e, ' ')
            bagtype = e_split[1] + tf.constant(' ') + e_split[2]
            k = index_map[bagtype]

            if k > -1:
                amt = tf.strings.to_number(e_split[0])
                indices = indices.write(ta_i, (i, k))
                updates = updates.write(ta_i, amt)
                ta_i += 1

    return tf.scatter_nd(indices.stack(), updates.stack(), (n, n)), sg_idx

@tf.function(experimental_compile=True)
def solve1(X, sg_idx):
    X = tf.transpose(X)
    X_orig = X
    res_vec = tf.zeros(tf.shape(X)[0])
    while tf.math.count_nonzero(X) > 0:
        res_vec += X[sg_idx]
        X = X@X_orig
    return tf.math.count_nonzero(res_vec)

@tf.function(experimental_compile=True)
def solve2(X, sg_idx):
    X_orig = X
    res_vec = 0.0
    while tf.math.count_nonzero(X) > 0:
        res_vec += tf.reduce_sum(X[sg_idx])
        X = X@X_orig
    return res_vec

In [5]:
%%time
X, sg_idx = extract_matrix(tf.io.read_file('day7.txt'))

CPU times: user 6.32 s, sys: 1.03 s, total: 7.35 s
Wall time: 2.31 s


In [6]:
%%timeit
solve1(X, sg_idx)

2.9 ms ± 79.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
solve2(X, sg_idx)

2.77 ms ± 92.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
